In [41]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import sounddevice as sd

In [42]:
detector = HandDetector(detectionCon=0.8, maxHands=2)

In [43]:
GESTURE = {
    "bass": [0, 0, 0, 0, 0],
    "mid": [0, 1, 0, 0, 0],
    "treble": [0, 0, 0, 0, 1],
    "all": [1, 1, 1, 1, 1],
    "toggle": [1, 1, 0, 0, 1]  # gesture to turn on/off adjustment mode
}

In [44]:
EQ_RANGE = [0, 100]
VOLUME_RANGE = (-50, 50)
INITIAL_VOLUME_ANGLE = 60

In [45]:
audio_file = 'playlist\Gusto.mp3'

In [46]:
import os
if os.path.exists(audio_file):
    print("File exists!")
else:
    print("File not found.")

File exists!


In [47]:
from pydub.utils import which
print(which("ffmpeg"))


C:\Users\ADMIN\ffmpeg\ffmpeg-2025-05-01-git-707c04fe06-full_build\bin\ffmpeg.exe


In [48]:
from EqController import EQController
from AudioPlayer import AudioPlayer

eq_controller = EQController()
player = AudioPlayer(audio_file, eq_controller)

adjust_mode = False
prev_toggle_state = False  # to prevent flipping too fast
toggle_cooldown = 0


In [49]:
import threading
threading.Thread(target=player.play).start()

In [50]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    if not success:
        break

    hands, img = detector.findHands(img)

    if len(hands) == 2:
        hand1, hand2 = hands
        if hand1["center"][0] < hand2["center"][0]:
            left_hand = hand2
            right_hand = hand1
        else:
            left_hand = hand1
            right_hand = hand2

        fingers_left = detector.fingersUp(left_hand)
        # == Adjustment Mode == 
        if fingers_left == GESTURE["toggle"]:
            if not prev_toggle_state and toggle_cooldown == 0:
                adjust_mode = not adjust_mode
                toggle_cooldown = 20 # frames to prevent spamming toggle
            prev_toggle_state = True
        else:
            prev_toggle_state = False
        
        if toggle_cooldown > 0:
            toggle_cooldown -= 1

        freq_band = "none" # default
        gain = 1.0 # default
        volume = 1.0 # default
        if adjust_mode:
            for control, gesture in GESTURE.items():
                if fingers_left == gesture:
                    freq_band = control

                    if control == "all": # adjust the volume
                        wrist = np.array(right_hand['lmList'][0])
                        middle_tip = np.array(right_hand['lmList'][12])

                        dx = middle_tip[0] - wrist[0]
                        dy = middle_tip[1] - wrist[1]
                        angle = -math.degrees(math.atan2(dy, dx)) - INITIAL_VOLUME_ANGLE
                        angle_clamped = max(
                            VOLUME_RANGE[0], min(VOLUME_RANGE[1], angle))
                        # gain = int(angle_clamped)
                        volume = (angle_clamped - VOLUME_RANGE[0]) / (VOLUME_RANGE[1] - VOLUME_RANGE[0])
                        # set band name 
                        # eq_controller.set_band(control)
                        # eq_controller.set_gain(gain)
                        player.set_volume(volume)
                        # assign band name and gain
                        # freq_band = eq_controller.get_selected_band()
                        # gain = eq_controller.get_gains()
                        player.get_volume()
                        

                    else: # adjust the eq
                        distance, info, img = detector.findDistance(
                            right_hand["lmList"][8][:-1],
                            right_hand["lmList"][4][:-1],
                            img
                        )
                        gain = max(EQ_RANGE[0], min(EQ_RANGE[1], int(distance * 0.5)))
                        # set band name 
                        eq_controller.set_band(control)
                        eq_controller.set_gain(gain)
                        # assign band name and gain
                        freq_band = eq_controller.get_selected_band()
                        gain = eq_controller.get_gains()
                    break    

        cv2.putText(img, f"Freq Band: {freq_band}", (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Gain: {gain}", (10, 110),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Volume: {int(volume*100)} %", (10, 150),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Adjust Mode: {'ON' if adjust_mode else 'OFF'}", (10, 190),
            cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0) if adjust_mode else (0, 0, 255), 3)

    cv2.imshow("Hand Gesture Equalizer Control", img)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        player.stop()
        break
    
cap.release()
cv2.destroyAllWindows()